In [1]:
source_filepath = 'source\question'
input_filepath = 'input\question'
output_filepath = 'output'
font_filepath = 'source'

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from scipy.stats import chi2_contingency
import warnings
import seaborn as sns
import re
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import datetime
from math import sqrt

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
#from keras.layers.core import Dense, Dropout, Activation           ## Lib for NN (not used)
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
warnings.filterwarnings('ignore')

In [5]:
plt.rcParams.update({'font.size': 22})
fonts = font_filepath+'/THSarabunNew.ttf'
prop = fm.FontProperties(fname=fonts)# Call Font file that can support Thai Language

# Read file

In [6]:
def read_file():
    external = pd.read_excel(input_filepath+'/External_each_year_sector.xlsx')
    external.drop(columns=['Unnamed: 0.1','Unnamed: 0','Date','Month','Sales_group_code'],inplace = True)
    external = external.groupby(['Year','Sector']).mean()
    question = pd.read_csv(input_filepath+'/Flag_engage_pivot_with_revenue.csv')
    Question_External = pd.merge(question,external,left_on=['Year','SECTOR'],right_on=['Year','Sector'],how = 'inner')
    return Question_External

# Preprocess for Model

อ้างอิงจากเอกสาร บทที่ 4 ข้อ 6<br>
<blockquote>การเลื่อนข้อมูลเป็นหนึ่งในเทคนิคสำหรับการจัดเตรียมข้อมูลเพื่อสร้างแบบจำลองที่ต้องการใช้ข้อมูลในอดีต โดยมีหลักการทำงานหลัก ๆ คือการเลื่อนข้อมูลทั้งหมดที่ต้องการตามแนวแถวไปยังช่วงเวลาที่ต้องการ ตัวอย่างเช่น ตารางที่ 1 หากต้องเลื่อนข้อมูลของวันที่ 1/01/63 มาเป็นข้อมูลของวันที่1/03/63 สามารถทำได้โดยการเลื่อนข้อมูลทั้งหมด 2 ขั้น และทำการสร้างข้อมูลเป็นคอลัมน์ใหม่</blockquote>
<img src="pics/AA.png" width="180">

In [7]:
def preprocessing(df):
    dict_ = pd.read_excel(source_filepath+'/Dict.xlsx')
    dict_.dropna(inplace = True)
    temp = pd.Series(dict_['Unnamed: 2'].values,index=dict_.Q_CD).to_dict()
    df.rename(columns = temp,inplace = True)
    df.drop(columns =['Unnamed: 0'],inplace = True)
    df.rename(columns={'ยอดเงินรวม': 'Revenue'}, inplace=True)
    
    data = df.drop(columns=['SECTOR','Overall SAT'])
    one_hot = pd.get_dummies(data['BIZ_TYP_DESC2'])
    data = data.drop('BIZ_TYP_DESC2',axis = 1)
    data = data.join(one_hot)
    data = data.fillna(0)

    data_2015 = data[data.Year == 2015] 
    data_2016 = data[data.Year == 2016] 
    data_2017 = data[data.Year == 2017] 
    data_2018 = data[data.Year == 2018]

    target = df[['Overall SAT','Year','SHIP_TO']]
    target_2015 = target[target.Year == 2015] 
    target_2016 = target[target.Year == 2016] 
    target_2017 = target[target.Year == 2017]
    target_2018 = target[target.Year == 2018]

    df1 = data_2015.merge(target_2016,on='SHIP_TO')
    df2 = data_2016.merge(target_2017,on='SHIP_TO')
    df3 = data_2017.merge(target_2018,on='SHIP_TO')
    df = pd.concat([df1, df2, df3])

    data = df.drop(columns = ['Overall SAT','SHIP_TO','Year_x','Year_y'])
    target = df[['SHIP_TO','Overall SAT']]
    target.rename(columns={'Overall SAT': 'Actual'}, inplace=True)

    column_names = list(data.columns.values)

    return data,target,column_names

In [8]:
def preprocessing_modified(df):
    dict_ = pd.read_excel(source_filepath+'/Dict.xlsx')
    dict_.dropna(inplace = True)
    temp = pd.Series(dict_['Unnamed: 2'].values,index=dict_.Q_CD).to_dict()
    df.rename(columns = temp,inplace = True)
    df.drop(columns =['Unnamed: 0'],inplace = True)
    df.rename(columns={'ยอดเงินรวม': 'Revenue'}, inplace=True)
    
    data = df.drop(columns=['SECTOR','Overall SAT'])
    one_hot = pd.get_dummies(data['BIZ_TYP_DESC2'])
    data = data.drop('BIZ_TYP_DESC2',axis = 1)
    data = data.join(one_hot)
    data = data.fillna(0)

    data_2015 = data[data.Year == 2015] 
    data_2016 = data[data.Year == 2016] 
    data_2017 = data[data.Year == 2017] 
    data_2018 = data[data.Year == 2018]
    data_2019 = data[data.Year == 2019]

    target = df[['Overall SAT','Year','SHIP_TO']]
    target_2015 = target[target.Year == 2015] 
    target_2016 = target[target.Year == 2016] 
    target_2017 = target[target.Year == 2017]
    target_2018 = target[target.Year == 2018]
    target_2019 = target[target.Year == 2019]

    df1 = data_2015.merge(target_2016,on='SHIP_TO')
    df2 = data_2016.merge(target_2017,on='SHIP_TO')
    df3 = data_2017.merge(target_2018,on='SHIP_TO')
    df4 = data_2018.merge(target_2019,on='SHIP_TO')
    df = pd.concat([df1, df2, df3, df4])
    
    #df1 = data_2015.merge(target_2015,on='SHIP_TO')
    #df2 = data_2016.merge(target_2016,on='SHIP_TO')
    #df3 = data_2017.merge(target_2017,on='SHIP_TO')
    #df4 = data_2018.merge(target_2018,on='SHIP_TO')
    #df5 = data_2019.merge(target_2019,on='SHIP_TO')
    #df = pd.concat([df1, df2, df3, df4, df5])

    data = df.drop(columns = ['Overall SAT','SHIP_TO','Year_x','Year_y'])
    target = df[['SHIP_TO','Overall SAT']]
    target.rename(columns={'Overall SAT': 'Actual'}, inplace=True)
    
    data_latest_year = data_2019.drop(columns = ['SHIP_TO','Year'])
    target_latest_year = target_2019.drop(columns = ['Overall SAT','Year'])

    return data,target,data_latest_year,target_latest_year

# Random Forest Regression

In [9]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'min_samples_leaf': [3, 4, 5, 6],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

In [10]:
def Random_Forest_Regression(data, target, param_grid):
    rnd_clf = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=42)
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)
    X_train, X_test, y_train, y_test = train_test_split(data,target, test_size=0.33, random_state=42)
    y_train.drop(columns = ['SHIP_TO'],inplace = True)
    grid_search = GridSearchCV(estimator = rnd_clf, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 2)
    grid_search.fit(X_train, y_train)
    print(grid_search.best_params_)
    return grid_search,X_test,y_test

In [11]:
def Evaluate(model,X_test,y_test):
    best_grid = model.best_estimator_
    predictions = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test['Actual'],predictions))
    print('The best model evaluate:',rmse)
    return rmse,best_grid,predictions

In [12]:
def Predict(model,X_test,y_test):
    best_grid = model.best_estimator_
    predictions = model.predict(X_test)
    return predictions

In [13]:
def show_important_feature(column_names,best_grid):
    features = list(column_names)
    importances = best_grid.feature_importances_
    indices = np.argsort(importances)[::-1][:5]
    plt.figure(figsize=(25,5))
    plt.title('Feature Importances')
    plt.bar(range(len(indices)), importances[indices], color='g', align='center')
    plt.xticks(range(len(indices)), [features[i] for i in indices], rotation=90, fontproperties=prop)
    plt.xlabel('Relative Importance')
    plt.show()

## Running train-test model 2015-2017

In [14]:
Question_External = read_file()
#data, target, col_name = preprocessing(Question_External)  #2015-2017
data,target,data_latest,target_latest = preprocessing_modified(Question_External) #2015-2018

In [15]:
#print('Running Random Forest Regression ...')
# Random Forest Regression
model, X_test,y_test = Random_Forest_Regression(data,target,param_grid)
rmse,best_grid,predictions = Evaluate(model,X_test,y_test)
result = pd.DataFrame({'RMSE': [rmse]})
#y_test['Predict'] = predictions
#result.to_csv(output_filepath+'\OverAllSAT_Randomforest_evaluation_'+str(datetime.datetime.now().strftime('%Y%m%d_%H%M'))+'.csv') 
#y_test.to_csv(output_filepath+'\OverallSAT_Randomforest_prediction_'+str(datetime.datetime.now().strftime('%Y%m%d_%H%M'))+'.csv')

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   39.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed: 17.3min finished


{'bootstrap': True, 'max_depth': 90, 'min_samples_leaf': 3, 'min_samples_split': 12, 'n_estimators': 1000}
The best model evaluate: 0.7403459234029068


In [16]:
result

,RMSE
0,0.740346


In [17]:
#show_important_feature(column_names,best_grid)

## Result 2019 with best param in model evaluation process

In [27]:
best_grid

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=90,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=3, min_samples_split=12,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [24]:
# กำหนดค่าจาก Hyperparameter ลง Dictionary ด้วย Best param (ตัวแปร best_grid)
#param_best_grid = {
#    'bootstrap':True, 
#    'max_depth':110,
#    'min_samples_leaf':3, 
#    'min_samples_split':12,
#    'n_estimators':1000
#}

In [25]:
#Question_External = read_file()
#data,target,data_latest,target_latest = preprocessing_modified(Question_External)

In [26]:
#scaler = MinMaxScaler()
#data = scaler.fit_transform(data)
#rnd_Reg = RandomForestRegressor(n_estimators=param_best_grid.get("n_estimators"),
#                                max_depth=param_best_grid.get("max_depth"),
#                                min_samples_leaf=param_best_grid.get("min_samples_leaf"),
#                                min_samples_split=param_best_grid.get("min_samples_split"))
#rnd_Reg.fit(data, target)

In [28]:
predict_val = model.predict(data_latest)
target_latest['YEAR'] = 2019
target_latest['PREDICT'] = predict_val

In [29]:
target_latest

,SHIP_TO,YEAR,PREDICT
1155,30000001.0,2019,8.979658
1156,30000009.0,2019,8.979658
1157,30000010.0,2019,8.944455
1158,30000083.0,2019,8.919968
1159,30000136.0,2019,8.906017
1160,30000137.0,2019,8.981746
1161,30000165.0,2019,8.906017
1162,30015470.0,2019,8.936909
1163,30018793.0,2019,8.906017
1164,30021634.0,2019,8.905946


In [32]:
target_latest.to_csv(output_filepath+'\OverallSat_prediction_2019.csv',index=False)